In [1]:
from __future__ import division, print_function

import tensorflow as tf
import numpy as np
import h5py
import scipy.io

In [2]:
from tensorflow.contrib import rnn

In [ ]:
def fix_hdf5():
    import gc
    for obj in gc.get_objects():
        if isinstance(obj, h5py.File):
            try:
                obj.close()
            except:
                pass

In [ ]:
trainmat.close()

In [3]:
np.random.seed(1337)

# Load the data into python

In [4]:
trainmat = h5py.File('data/deepsea_train/train.mat', "r")
validmat = scipy.io.loadmat('data/deepsea_train/valid.mat')
testmat = scipy.io.loadmat('data/deepsea_train/test.mat')

In [5]:
X_train = np.transpose(np.array(trainmat['trainxdata']), axes=(2,0,1))

In [12]:
print(X_train.shape)
X_train[1:2]

(4400000, 1000, 4)


array([[[0, 0, 0, 0],
        [0, 0, 0, 0],
        [0, 0, 0, 0],
        ..., 
        [1, 0, 0, 0],
        [0, 1, 0, 0],
        [1, 0, 0, 0]]], dtype=uint8)

In [6]:
y_train = np.transpose(trainmat['traindata']).T

In [11]:
print(y_train.shape)
y_train[1:2]

(919, 4400000)


array([[0, 0, 0, ..., 0, 1, 0]], dtype=uint8)

In [7]:
learning_rate = 0.001
num_steps = 2000
batch_size = 1000

input_dim = 4
input_length = 1000
num_classes = 919
dropout_cnn = 0.2

# CNN 

In [25]:
def BDNN(x):
    forward_lstm = rnn.BasicLSTMCell(320)
    backward_lstm = rnn.BasicLSTMCell(320)
    
    brnn, _, _ = tf.nn.static_bidirectional_rnn(forward_lstm, backward_lstm, tf.unstack(x), dtype=tf.float32)
    print(brnn)
    return brnn

def DanQNN(x, input_length, nb_filter, filter_length,
            activation=tf.nn.relu, subsample=1, border="VALID", reuse=False, max_strides=13,
          max_pool_size=13):
    
    '''
    width = 26
    stride = 1?
    '''
    
    with tf.variable_scope('DanQNN', reuse=reuse):
    
        conv1d = tf.layers.conv1d(x, filters=nb_filter , strides=subsample, 
                                  padding=border, kernel_size=input_length, data_format='channels_first', reuse=True)
        print(conv1d)
        max1 = tf.layers.max_pooling1d(conv1d, pool_size=max_pool_size, strides=max_strides)
        
        max1 = tf.layers.dropout(max1, rate=0.2)
        
        print(max1)
        brnn = BDNN(max1)
        
        brnn = tf.layers.dropout(brnn, rate=0.5)
        
        brnn = tf.contrib.layers.flatten(brnn)

        #fc = ft.layers.dense(brnn, 75*640)
        print(brnn)
        return brnn
        

In [26]:
DanQNN(tf.cast(testmat['testxdata'][1:10000], dtype=tf.float32),1000,320,26,)


Tensor("DanQNN_9/conv1d/add:0", shape=(9999, 320, 1), dtype=float32)
Tensor("DanQNN_9/dropout/Identity:0", shape=(9999, 24, 1), dtype=float32)
[<tf.Tensor 'DanQNN_9/concat:0' shape=(24, 640) dtype=float32>, <tf.Tensor 'DanQNN_9/concat_1:0' shape=(24, 640) dtype=float32>, <tf.Tensor 'DanQNN_9/concat_2:0' shape=(24, 640) dtype=float32>, <tf.Tensor 'DanQNN_9/concat_3:0' shape=(24, 640) dtype=float32>, <tf.Tensor 'DanQNN_9/concat_4:0' shape=(24, 640) dtype=float32>, <tf.Tensor 'DanQNN_9/concat_5:0' shape=(24, 640) dtype=float32>, <tf.Tensor 'DanQNN_9/concat_6:0' shape=(24, 640) dtype=float32>, <tf.Tensor 'DanQNN_9/concat_7:0' shape=(24, 640) dtype=float32>, <tf.Tensor 'DanQNN_9/concat_8:0' shape=(24, 640) dtype=float32>, <tf.Tensor 'DanQNN_9/concat_9:0' shape=(24, 640) dtype=float32>, <tf.Tensor 'DanQNN_9/concat_10:0' shape=(24, 640) dtype=float32>, <tf.Tensor 'DanQNN_9/concat_11:0' shape=(24, 640) dtype=float32>, <tf.Tensor 'DanQNN_9/concat_12:0' shape=(24, 640) dtype=float32>, <tf.Tensor

Tensor("DanQNN_9/Flatten/Reshape:0", shape=(9999, 15360), dtype=float32)


<tf.Tensor 'DanQNN_9/Flatten/Reshape:0' shape=(9999, 15360) dtype=float32>

In [22]:
testmat['testxdata'][1:100].shape

(99, 4, 1000)

In [ ]:
def model_fn(features, labels, mode):

    logits_train = DanQNN(features, num_classes, reuse=False)
    logits_test = DanQNN(features, num_classes, reuse=True)
    
    pred_classes = tf.argmax(logits_test, axis=1)
    pred_probas = tf.nn.softmax(logits_test)
    
    if mode == tf.estimator.ModeKeys.PREDICT:
        return tf.estimator.EstimatorSpec(mode, predictions=pred_classes)
    
    loss_op = tf.reduce_mean(tf.nn.sparse_softmax_cross_entropy_with_logits(
        logits=logits_train, labels=tf.cast(labels, dtype=tf.int32)))
    

In [ ]:
# initialise the weights
weights = {'w1' : tf.Variable(tf.random_normal([])),
           'out' : tf.Variable(tf.random_normal([]))}

# initalise biase with size of output 
biases = {'b1' : tf.Variable(tf.random_normal([])),
         'out' : tf.Variable(tf.random_normal([]))}

# Construct model

logits = conv_net(X_train, weights, biases, dropout)
prediction = tf.nn.sigmoid(logits)

# Define loss and optimizer
loss_op = tf.reduce_mean(tf.nn.sigmoid_cross_entropy_with_logits(logits=logits, labels=y_train))
optimiser = tf.train.AdamOptimizer(learning_rate=learning_rate)
train_op = optimiser.minimize(loss_op)

# Evaluate model

correct_pred = tf.equal(tf.argmax(prediction, 1), tf.argmax(y_train, 1))
accuracy = tf.reduce_mean(tf.cast(correct_pred, tf.float32))

# initialise the variables

init = tf.global_variables_initializer()

In [ ]:
# mini example of 1D convolutional NN

import tensorflow as tf

i = tf.constant([10, 50, 60, 10, 20, 40, 30], dtype=tf.float32, name='i')
k = tf.constant([.3, .3, .3], dtype=tf.float32, name='k')

data   = tf.reshape(i, [1, int(i.shape[0]), 1], name='data')
print(data)
kernel = tf.reshape(k, [int(k.shape[0]), 1, 1], name='kernel')
print(kernel)
res = tf.squeeze(tf.nn.conv1d(data, kernel, 1, 'SAME'))
with tf.Session() as sess:
    print(sess.run(i))
    print(sess.run(k))
    print(sess.run(data))
    print(sess.run(kernel))
    print(sess.run(res))

In [ ]:
from keras import backend as K
from keras.utils.conv_utils import convert_kernel
import tensorflow as tf
ops = []
for layer in model.layers:
   if layer.__class__.__name__ in ['Convolution1D', 'Convolution2D', 'Convolution3D', 'AtrousConvolution2D']:
      original_w = K.get_value(layer.W)
      converted_w = convert_kernel(original_w)
      ops.append(tf.assign(layer.W, converted_w).op)

In [ ]:
tf.layers.conv1d